In [1]:
import numpy as np
import pandas as pd
import datatable
import itertools as it
from owlready2 import *
import math
import joblib

import pickle

#para depurar los que letras que no están en el codigo ascii
import unicodedata
import functools
import spacy
import stanfordnlp
#from spacy_stanfordnlp import StanfordNLPLanguage
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import metrics
#from wiki_dump_reader import Cleaner, iterate
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize,scale

#from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
#from gensim.models.wrappers import LdaMallet
#from gensim.corpora import Dictionary

#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import SVG

import random
from sklearn.externals import joblib


#Operaciones con gráficos
import scattertext as st
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

import networkx as nx

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool, LabelSet, ColumnDataSource, Range1d
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE

#import gensim
#import gensim.corpora as corpora
#from gensim.models import CoherenceModel
#from gensim.models import HdpModel

warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from multiprocessing import  Pool
import math
import scipy.sparse as sp

import vectorizer
importlib.reload(vectorizer)
from vectorizer import  WeightedTfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import *
from sklearn.metrics import classification_report, confusion_matrix



# Reorganizamos el corpus

In [2]:
import joblib
corpus=joblib.load('../data/processed/20newsgroup/corpus_reordered.final.pck')

In [3]:
corpus

,cleaned_text,Target,Topic,prob_topic,lof,outlier_iqr,new_target
0,organization post office carnegie mellon pitts...,0,1,0.997507,0,False,1.0
1,summary seek recommendation vlb video card nnt...,1,1,0.997566,0,False,1.0
2,line 95 nntp post host viktoria.dsv.su.se orga...,0,0,0.001600,0,False,0.0
3,originator guyd pal500.austin.ibm.com organiza...,1,1,0.997804,0,False,1.0
4,organization sophomore mechanical engineer car...,0,1,0.997406,0,False,1.0
...,...,...,...,...,...,...,...
18841,distribution world organization invention fact...,0,0,0.002171,0,False,0.0
18842,organization t distribution na line 21 article...,0,1,0.997347,0,False,1.0
18843,organization mail group x newsreader tin versi...,1,1,0.996557,0,False,1.0
18844,organization central line 27 nntp post host hc...,0,1,0.997770,0,False,1.0


In [13]:
corpus.head()

,cleaned_text,Target,Topic,prob_topic,lof,outlier_iqr,new_target
0,organization post office carnegie mellon pitts...,0,1,0.997507,0,False,1.0
1,summary seek recommendation vlb video card nnt...,1,1,0.997566,0,False,1.0
2,line 95 nntp post host viktoria.dsv.su.se orga...,0,0,0.001600,0,False,0.0
3,originator guyd pal500.austin.ibm.com organiza...,1,1,0.997804,0,False,1.0
4,organization sophomore mechanical engineer car...,0,1,0.997406,0,False,1.0


In [14]:
corpusnormalizado = corpus.iloc[:,[0,6]]
corpusnormalizado.head()

,cleaned_text,new_target
0,organization post office carnegie mellon pitts...,1.0
1,summary seek recommendation vlb video card nnt...,1.0
2,line 95 nntp post host viktoria.dsv.su.se orga...,0.0
3,originator guyd pal500.austin.ibm.com organiza...,1.0
4,organization sophomore mechanical engineer car...,1.0


In [4]:
corpussinoutliers = corpus.iloc[:,[0,2]]
corpussinoutliers.head()

,cleaned_text,Topic
0,organization post office carnegie mellon pitts...,1
1,summary seek recommendation vlb video card nnt...,1
2,line 95 nntp post host viktoria.dsv.su.se orga...,0
3,originator guyd pal500.austin.ibm.com organiza...,1
4,organization sophomore mechanical engineer car...,1


In [6]:
topics0 = joblib.load('../objects/20newsgroup/topics0')
topics1 = joblib.load('../objects/20newsgroup/topics1')
feature_names = joblib.load('../objects/20newsgroup/feature_names.final')

In [7]:
longitud = 100
vocab=topics1[-100:]
vocab2 = topics0[-200:]
contador = 0
for word in vocab2:
    if word not in vocab:
        contador = contador+1
        vocab.append(word)
        if contador == longitud:
            break

In [8]:
vocab

['read',
 '24',
 'service',
 'numb',
 'ask',
 'dept',
 'question',
 'news',
 'support',
 '17',
 '25',
 'college',
 'inc',
 'message',
 'net',
 'etc',
 'engineer',
 'set',
 'well',
 '13',
 'address',
 'bite',
 '20',
 'think',
 'available',
 'good',
 'newsreader',
 'list',
 'ca',
 'get',
 'price',
 'sell',
 'space',
 'include',
 'time',
 'machine',
 'cs',
 'buy',
 'fax',
 '14',
 'interest',
 'datum',
 'send',
 '11',
 'state',
 'don',
 'science',
 'phone',
 'technology',
 'advance',
 'key',
 'appreciate',
 '12',
 'want',
 '15',
 'try',
 '16',
 'uk',
 'access',
 'internet',
 'find',
 'keyword',
 'world',
 'information',
 'help',
 'hello',
 'email',
 '10',
 'version',
 'software',
 'look',
 'usa',
 'problem',
 'need',
 'run',
 'article',
 'like',
 'card',
 'new',
 'program',
 'work',
 'computer',
 'write',
 'have',
 'drive',
 'use',
 'know',
 'file',
 'be',
 'system',
 'mail',
 'distribution',
 'com',
 'thank',
 'window',
 'nntp',
 'host',
 'university',
 'post',
 'edu',
 'department',
 'ok

In [10]:
wtfidf = WeightedTfidfVectorizer(vocabulary = vocab) #aquí ya hemos hecho una selección inicial de variables

In [8]:
newdtm = wtfidf.fit_transform(corpusnormalizado)

In [9]:
newdtm

<18846x200 sparse matrix of type '<class 'numpy.float32'>'
	with 402284 stored elements in Compressed Sparse Row format>

In [11]:
newdtmso = wtfidf.fit_transform(corpussinoutliers)

In [12]:
newdtmso

<18846x200 sparse matrix of type '<class 'numpy.float32'>'
	with 402284 stored elements in Compressed Sparse Row format>

In [10]:
joblib.dump(vocab,'../objects/20newsgroup/vocab.dtm.final')
joblib.dump(newdtm,'../objects/20newsgroup/newdtm.final')

['../objects/20newsgroup/newdtm.final']

In [15]:
joblib.dump(corpusnormalizado,'../data/processed/20newsgroup/corpus_normalizado.final')

NameError: name 'corpusnormalizado' is not defined

In [18]:
joblib.dump(corpussinoutliers,'../data/processed/20newsgroup/corpus_normalizado_sin_outliers.final')

['../data/processed/20newsgroup/corpus_normalizado_sin_outliers.final']

In [17]:
joblib.dump(newdtmso,'../objects/20newsgroup/newdtmsinoutliers.final')

['../objects/20newsgroup/newdtmsinoutliers.final']

In [17]:
corpusnormalizado.head()

,cleaned_text,new_target
0,organization post office carnegie mellon pitts...,1.0
1,summary seek recommendation vlb video card nnt...,1.0
2,line 95 nntp post host viktoria.dsv.su.se orga...,0.0
3,originator guyd pal500.austin.ibm.com organiza...,1.0
4,organization sophomore mechanical engineer car...,1.0
